In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras import datasets, layers, models
from keras.models import Sequential, Model
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense, Dropout
from keras.layers import Flatten
from keras.optimizers import SGD
from keras.utils import to_categorical
import matplotlib.pyplot as plt
import os
import sys
import datetime
from keras.regularizers import l2
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
import pandas as pd

In [2]:
%load_ext tensorboard

In [3]:
def load_dataset():
    # load cifar10 dataset
    (trainX, trainy), (testX, testy)  = datasets.cifar10.load_data()
    # summarize loaded dataset
    print('Train: X=%s, y=%s' % (trainX.shape, trainy.shape))
    print('Test: X=%s, y=%s' % (testX.shape, testy.shape))
     # one hot encode target values
    trainY = to_categorical(trainy)
    testY = to_categorical(testy)
    return trainX, trainY, testX, testY

In [4]:
# scale pixels
def prep_pixels(train, test):
    # convert from integers to floats
    train_norm = train.astype('float32')
    test_norm = test.astype('float32')
    # normalize to range 0-1
    train_norm = train_norm / 255.0
    test_norm = test_norm / 255.0
    # return normalized images
    return train_norm, test_norm

In [5]:
def separable_conv(i, ch):
    x = layers.DepthwiseConv2D((3,3), padding="same")(i)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)
    x = layers.Conv2D(ch, (1,1), padding="same")(x)
    x = layers.BatchNormalization()(x)
    return layers.Activation("relu")(x)

In [36]:
def define_model():
    input = layers.Input((32,32,3))
    x = layers.Conv2D(16, (3, 3), padding='same')(input)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)
    x = separable_conv(x, 24)
    x = layers.MaxPooling2D((2, 2))(x)
    x = separable_conv(x, 48)
    x = layers.MaxPooling2D((2, 2))(x)
    x = separable_conv(x, 96)
    x = separable_conv(x, 192)
    x = layers.MaxPooling2D((2, 2))(x)
    x = separable_conv(x, 96)
    x = separable_conv(x, 192)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Dense(500, activation='relu')(x)
    x = layers.Flatten()(x)
    x = layers.Dropout(0.2)(x)
    x = layers.Dense(10, activation='softmax')(x)
    model = Model(input, x)
    # compile model
    opt = SGD(learning_rate=0.001, momentum=0.9)
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    
    print(model.summary())

    return model

In [37]:
# plot diagnostic learning curves
def summarize_diagnostics(history, file):
    # plot loss
    plt.subplot(211)
    plt.title('Cross Entropy Loss')
    plt.plot(history.history['loss'], color='blue', label='train')
    plt.plot(history.history['val_loss'], color='orange', label='test')
    # plot accuracy
    plt.subplot(212)
    plt.title('Classification Accuracy')
    plt.plot(history.history['accuracy'], color='blue', label='train')
    plt.plot(history.history['val_accuracy'], color='orange', label='test')
    # save plot to file
    filename = sys.argv[0].split('/')[-1]
    plt.savefig(r"../logs/" + filename + file)
    plt.close()

In [42]:
def train_steps():
    # load dataset
    trainX, trainY, testX, testY = load_dataset()
    # prepare pixel data
    trainX, testX = prep_pixels(trainX, testX)
    # define model
    model = define_model()
    #ensures that logs are created and stored.
    #remove any previous logs
    print(model.summary())
    %rm -rf ./logs/
    log_dir = "../logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

    # We'll stop training if no improvement after some epochs
    earlystopper = EarlyStopping(monitor='val_accuracy', patience=8, verbose=1)
    reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5, patience=8, 
                                   verbose=1, mode='max', min_lr=0.00001)
    # fit model
    history = model.fit(trainX, trainY, epochs=100, batch_size=48, validation_split=0.20, callbacks=[earlystopper, reduce_lr, tensorboard_callback], verbose=1)
    print(model.summary())
    # evaluate model
    _, acc = model.evaluate(testX, testY, verbose=0)
    print('> %.3f' % (acc * 100.0))
    # learning curves
    summarize_diagnostics(history, 'ipykernel_launcher.py3blocks_depthconv_ver2.png')

    return model, testX, testY

In [39]:
model, testX, testY = train_steps()

Train: X=(50000, 32, 32, 3), y=(50000, 1)
Test: X=(10000, 32, 32, 3), y=(10000, 1)


Model: "model_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_11 (InputLayer)       [(None, 32, 32, 3)]       0         
                                                                 
 conv2d_93 (Conv2D)          (None, 32, 32, 16)        448       
                                                                 
 batch_normalization_124 (B  (None, 32, 32, 16)        64        
 atchNormalization)                                              
                                                                 
 activation_98 (Activation)  (None, 32, 32, 16)        0         
                                                                 
 depthwise_conv2d_49 (Depth  (None, 32, 32, 16)        160       
 wiseConv2D)                                                     
                                                                 
 batch_normalization_125 (B  (None, 32, 32, 16)        64  

In [40]:
model.summary()

Model: "model_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_11 (InputLayer)       [(None, 32, 32, 3)]       0         
                                                                 
 conv2d_93 (Conv2D)          (None, 32, 32, 16)        448       
                                                                 
 batch_normalization_124 (B  (None, 32, 32, 16)        64        
 atchNormalization)                                              
                                                                 
 activation_98 (Activation)  (None, 32, 32, 16)        0         
                                                                 
 depthwise_conv2d_49 (Depth  (None, 32, 32, 16)        160       
 wiseConv2D)                                                     
                                                                 
 batch_normalization_125 (B  (None, 32, 32, 16)        64  

In [43]:
def define_model():
    input = layers.Input((32,32,3))
    x = layers.Conv2D(32, (3, 3), padding='same')(input)
    x = layers.Conv2D(64, (3, 3), padding='same')(x)
    x = layers.Conv2D(128, (3, 3), padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)
    x = separable_conv(x, 24)
    x = layers.MaxPooling2D((2, 2))(x)
    x = separable_conv(x, 48)
    x = layers.MaxPooling2D((2, 2))(x)
    x = separable_conv(x, 96)
    x = separable_conv(x, 192)
    x = layers.MaxPooling2D((2, 2))(x)
    x = separable_conv(x, 96)
    x = separable_conv(x, 192)
    x = layers.MaxPooling2D((2, 2))(x)
    x = separable_conv(x, 96)
    x = separable_conv(x, 192)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Dense(500, activation='relu')(x)
    x = layers.Flatten()(x)
    x = layers.Dropout(0.2)(x)
    x = layers.Dense(10, activation='softmax')(x)
    model = Model(input, x)
    # compile model
    opt = SGD(learning_rate=0.001, momentum=0.9)
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    
    print(model.summary())

    return model

In [44]:
model, testX, testY = train_steps()

Train: X=(50000, 32, 32, 3), y=(50000, 1)
Test: X=(10000, 32, 32, 3), y=(10000, 1)


Model: "model_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_12 (InputLayer)       [(None, 32, 32, 3)]       0         
                                                                 
 conv2d_100 (Conv2D)         (None, 32, 32, 32)        896       
                                                                 
 conv2d_101 (Conv2D)         (None, 32, 32, 64)        18496     
                                                                 
 conv2d_102 (Conv2D)         (None, 32, 32, 128)       73856     
                                                                 
 batch_normalization_137 (B  (None, 32, 32, 128)       512       
 atchNormalization)                                              
                                                                 
 activation_111 (Activation  (None, 32, 32, 128)       0         
 )                                                         